In [1]:
#!pip3 freeze > requirements.txt

In [2]:
#install Ktrain
!pip install ktrain
!pip3 install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

     |████████████████████████████████| 25.3MB 116kB/s 
     |████████████████████████████████| 6.8MB 19.5MB/s 
     |████████████████████████████████| 983kB 32.7MB/s 
     |████████████████████████████████| 266kB 38.9MB/s 
     |████████████████████████████████| 1.9MB 28.7MB/s 
     |████████████████████████████████| 1.2MB 26.6MB/s 
     |████████████████████████████████| 471kB 40.8MB/s 
     |████████████████████████████████| 3.3MB 24.5MB/s 
     |████████████████████████████████| 901kB 26.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.4-cp37-none-any.whl size=25282461 sha256=db907924eadd5624a506eac018537f0fa322641daab71611af754323748fc63e
  Stored in directory: /root/.cache/pip/wheels/95/cc/1b/84c3799864cd864b99d94a8aeac2ee2d928fb3d0ec3d3f1a13
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993242 sha256=1709f36ebf8c2376890dbb34c6b991d7457e01d7d78cb69aeeb93fcde4a79b47
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7d

In [3]:
#Import required libraries
from google.colab import files
import pandas as pd
import io
import pandas as pd
import numpy as np
import ktrain


import tensorflow as tf
from ktrain import text
from google.colab import drive
from sklearn.model_selection import train_test_split
import os
import io
import requests
import shutil
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def load_dataset(url: str, sep: str = ',', decode: str = 'utf-8', keep_default_na: bool = True, error_bad_lines: bool = True):
    def requests_csv(url: str, decode: str):
        return (requests.get(url).content).decode(decode)
    
    return pd.read_csv(io.StringIO(requests_csv(url, decode)),
                       sep=sep,
                       keep_default_na=keep_default_na,
                       error_bad_lines=error_bad_lines
                       )
url_dataset = "https://media.githubusercontent.com/media/Y4rd13/datasets/main/upwork_dataset.csv"                      

In [5]:
df = load_dataset(url_dataset, decode="cp1252", keep_default_na=True, error_bad_lines=True)
df.head()

,title,jobRegion,jobText,jobExperienceLevel,jobType,clientLocation,clientRating,approved
0,Web Design & Development Team (on-going work),Worldwide,We need a team that is capable of working with...,Expert,Complex project,United States,5.0,1
1,Website development,Worldwide,We are looking for someone to build a website ...,Expert,One-time project,United States,5.0,1
2,Auction Website to sell Household items,Worldwide,I am looking to hire a Agency or freelancers w...,Expert,Complex project,United States,0.0,0
3,App development (Android & iOS),Worldwide,I'm looking for someone that can make a comple...,Intermediate,One-time project,Sweden,4.0,0
4,Mobile App Development (Media playback app),Worldwide,-What the deliverable is an audio player mobil...,Intermediate,One-time project,United States,5.0,0


In [6]:
def check_duplicated(df: pd.core.frame.DataFrame, subset: str, verbose: int = 0):
    """Check duplicated data in the subset of a dataframe.

    Parameters
    ----------
    df : pd.core.frame.DataFrame
        dataframe
    subset : str
        column to be checked
    verbose : int = 0
        0 or 1. Verbosity mode. 0 = silent, 1 = shape of the dataframe.
    
    Returns
    -------
    duplicated data info and preprocessing
    """
    check = df.duplicated(subset=subset)
    idx = 0
    if check.any() == False:
        print('No duplicates found')
    
    else:
        for i in check:
            if i == True:
                idx+=1

        print('Total of duplicates: {}'.format(idx))

    if verbose == 1:
        duplicates = idx
        total, __ = df.shape
        
        print('Dataframe shape: {}'.format(df.shape))
        
        if duplicates != 0:
            print('Dataframe without duplicates: {}'.format(total - duplicates))

In [7]:
# Lets check if the dataset have any duplicates with our utils function
check_duplicated(df, subset='jobText', verbose=True)

Total of duplicates: 28
Dataframe shape: (4384, 8)
Dataframe without duplicates: 4356


In [8]:
df.dtypes

title                  object
jobRegion              object
jobText                object
jobExperienceLevel     object
jobType                object
clientLocation         object
clientRating          float64
approved                int64
dtype: object

Checking version of TF, so that when later we will reload the model, we can use same version of TF:

In [9]:
#check tensorflow version
tf.version.VERSION

'2.5.0'

# Preparing our train and test dataset:

Get data loaded from train.txt to df_train dataframe and teast.txt to df_test dataframe.

In [10]:
df['approved'] = df['approved'].astype(str)
df['jobText'] = df['jobText'].astype(str)

In [11]:
#Splitting the data : train and val
df_train,df_test = train_test_split(df , test_size=0.2, random_state=42)

Checking first few rows of our train dataset.

In [12]:
df_train.head()

,title,jobRegion,jobText,jobExperienceLevel,jobType,clientLocation,clientRating,approved
647,Need to build an interactive app,Worldwide,Someone who understands how to build an intera...,Intermediate,Ongoing project,United States,0.00,1
2194,React Native Developer,Worldwide,I am looking to hire someone who has much expe...,Intermediate,Ongoing project,United States,4.99,0
2244,App developer needed for music collaboration a...,Worldwide,Im looking for a reliable skilled app develope...,Intermediate,One-time project,United States,0.00,1
4258,App for messengers,Worldwide,Looking to develope a cellphone App for messeg...,Intermediate,One-time project,Costa Rica,0.00,0
1164,Developer needed to build web app from ground up,Worldwide,We are looking for a developer to help us get ...,Expert,Ongoing project,United States,4.00,1


Checking category wise distribution of our test data.

In [13]:
#the unique value counts for the categories
df_train.approved.value_counts()

0    2097
1    1410
Name: approved, dtype: int64

In [14]:
X_train = df_train.jobText.tolist()
X_test = df_test.jobText.tolist()
y_train = df_train.approved.tolist()
y_test = df_test.approved.tolist()

Checking size of our train and test datasets:

In [15]:
print(len(X_train),len(X_test),len(y_train),len(y_test))

3507 877 3507 877


Our dataset has below categories/factors:

In [16]:
factors = [0,1]

Encoding our sentiment categories into numeric values:

In [17]:
encoding = { '0': 0,
    '1': 1
}

In [18]:
y_train = [encoding[key] for key in y_train]
y_test = [encoding[key] for key in y_test]

# Building Model using Transformer

We are using bert-base-uncased model. You can choose any other model. *maxlen of tokenization as 100 (512 it's max for BERT).*

In [19]:
model_arch ='bert-base-multilingual-uncased'
MAXLEN = 100
trans = text.Transformer(model_arch, maxlen=MAXLEN, class_names= factors)


Let's preprocess out test and train data set.

In [20]:
train_data = trans.preprocess_train(X_train,y_train)
test_data = trans.preprocess_test(X_test,y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 93
	95percentile : 281
	99percentile : 530


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 97
	95percentile : 293
	99percentile : 472


In [21]:
model = trans.get_classifier()

In [25]:
learner = ktrain.get_learner(model, train_data=train_data, val_data=test_data, batch_size=16)

Finding the best learning rate:

In [26]:
#learner.lr_find(show_plot=True, max_epochs=4)

In [ ]:
learner.fit_onecycle(1e-4, 5)
#learner.autofit(0.0001,10)



begin training using onecycle policy with max lr of 0.0001...
Epoch 1/5
220/220 [==============================] - 4297s 20s/step - loss: 0.6713 - accuracy: 0.5991 - val_loss: 0.6962 - val_accuracy: 0.5610
Epoch 2/5
220/220 [==============================] - 4289s 19s/step - loss: 0.6754 - accuracy: 0.5960 - val_loss: 0.7056 - val_accuracy: 0.5610
Epoch 3/5
 47/220 [=====>........................] - ETA: 53:04 - loss: 0.6879 - accuracy: 0.5652

**Confusion Matrix:**

In [ ]:
learner.model.summary()

In [ ]:
learner.validate(val_data=test_data, class_names=factors)

**top 5 data points not performing good:**

In [ ]:
learner.view_top_losses(n=5, preproc=trans)

In [ ]:
#X_test[1928]

Above data our model is predicting as sadness but label is mentioned as fear.

# Predict Data:

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=trans)

In [ ]:
inp = 'bad work vvv'

In [ ]:
predictor.predict(inp)

In [ ]:
predictor.explain(inp)